In [1]:
import datetime
import pandas as pd 

In [2]:
def DJI_to_UTC(time_str):
    year = int(time_str[:4])
    month = int(time_str[5:7])
    day = int(time_str[8:10])
    hour = int(time_str[11:13])
    minute = int(time_str[14:16])
    seconds = float(time_str[17:])
    microsecond = int(1e6 *  (seconds % 1))
    second = int(seconds - (seconds % 1))
    
    #print(year, month, day, hour, minute, seconds, second, microsecond)
    time_var = datetime.datetime(year, month, day, hour, minute, second, microsecond)
    
    epoch = datetime.datetime.utcfromtimestamp(0)
    utc_time = (time_var - epoch).total_seconds()
    return utc_time

def LoadFlightLog(filename):
    #Load the data as a pandas dataframe
    df = pd.read_csv('DJIFlightRecord_2019-12-17_[15-43-22]-TxtLogToCsv.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

    # remove the unnecessary columns from the dataframe
    header_list = list(df[:0])
    required_columns = ['CUSTOM.updateTime', 'OSD.flyTime [s]',  'OSD.latitude', 'OSD.longitude', 'OSD.height [m]', 'OSD.groundOrSky']
    for header in header_list:
        if(not(header in required_columns)):
            del df[header]

    # Add a UTC timestamp to the dataframe
    UTC_list = []
    for DJI_stamp in list(df['CUSTOM.updateTime']):
        #print(DJI_stamp)
        UTC_seconds = DJI_to_UTC(DJI_stamp)
        UTC_list.append(UTC_seconds)

    df['UTC_Seconds'] = UTC_list

    #overwrite the datatypes of dataframe
    data_types = df.astype({
                'CUSTOM.updateTime': 'string',
                'OSD.latitude': 'float64',
                'OSD.longitude': 'float64',
                'OSD.height [m]': 'float64',
                'OSD.groundOrSky': 'string',
                'OSD.flyTime [s]': 'float64',
                'UTC_Seconds': 'float64'
                }).dtypes

    df = df.astype(data_types)
    return df
'''
def GetDroneStatus(df, UTC):
    drone_status = df[df['UTC_Seconds'] <= UTC].iloc[-1]
    lat = drone_status['OSD.latitude']
    lon = drone_status['OSD.longitude']
    height = drone_status['OSD.height [m]']
    groundOrSky = drone_status['OSD.groundOrSky']
    Flytime = drone_status['OSD.flyTime [s]']
    Timestamp = drone_status['UTC_Seconds']
    return Timestamp, Flytime, lat, lon, height, groundOrSky

def GetStartUTC(df):
    return df.iloc[0]['UTC_Seconds']

def GetEndUTC(df):
    return df.iloc[-1]['UTC_Seconds']
'''

def GetDroneStatus(df, UTC):
    print(">>>>>>>> UTC:", UTC)
    drone_status = df[df['timestamp'] <= UTC].iloc[-1]
    lat = drone_status['latitude']
    lon = drone_status['longitude']
    height = drone_status['altitude(m)']
    groundOrSky = drone_status['isflying']
    Flytime = drone_status['time(millisecond)']
    Timestamp = drone_status['timestamp']
    return Timestamp, Flytime, lat, lon, height, groundOrSky

def LoadLitchiFlightLog(filename):
    #Load the data as a pandas dataframe
    df = pd.read_csv('2021-08-11_19-42-59_v2.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

    # remove the unnecessary columns from the dataframe
    header_list = list(df[:0])
    required_columns = ['timestamp', 'time(millisecond)',  'latitude', 'longitude', 'altitude(m)', 'isflying']
    for header in header_list:
        if(not(header in required_columns)):
            del df[header]

    #overwrite the datatypes of dataframe
    data_types = df.astype({
                'timestamp': 'float64',
                'time(millisecond)': 'float64',
                'latitude': 'float64',
                'longitude': 'float64',
                'altitude(m)': 'float64',
                'isflying': 'int',
                }).dtypes
    df = df.astype(data_types)

    #print(df.head())

    return df




In [6]:
filename = '2021-08-11_19-42-59_v2.csv'
data = LoadLitchiFlightLog(filename) # returns a pandas dataframe

#filename = 'DJIFlightRecord_2019-12-17_[15-43-22]-TxtLogToCsv.csv'
#data = LoadFlightLog(filename)
pd.options.display.float_format = '{:.6f}'.format
print(data.head())

   latitude  longitude  altitude(m)  time(millisecond)  isflying  \
0 51.999707  -0.454945     0.500000           0.000000         1   
1 51.999707  -0.454945     0.700000          89.000000         1   
2 51.999707  -0.454945     0.700000          97.000000         1   
3 51.999707  -0.454944     1.000000         200.000000         1   
4 51.999707  -0.454944     1.000000         207.000000         1   

             timestamp  
0 1628707379122.000000  
1 1628707379211.000000  
2 1628707379219.000000  
3 1628707379322.000000  
4 1628707379329.000000  


In [4]:
current_UTC = 1628707379329.000000
current_UTC = 1628707379123.101
GetDroneStatus(df = data, UTC = current_UTC)

>>>>>>>> UTC: 1628707379123.101


(1628707379122.0, 0.0, 51.999707, -0.454945, 0.5, 1.0)

In [5]:
start = GetStartUTC(data)
end = GetEndUTC(data)
print("Start UTC [s]:", start)
print("END UTC [s]:", end)
print("Duration [s]:", end-start)

NameError: name 'GetStartUTC' is not defined

In [ ]:
data.shape